In [17]:
'''讀取"是人臉"pgm影像，並轉換成一維樣本集'''

#導入函式庫
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

img_face_10304x200 = np.zeros(shape = (1,10304),dtype=int)   #建立要放入PCA的array

for folder_face_idx in range(1,41):      #開啟人臉圖片資料夾1~40
    for img_face_idx in range(1,11):     #選取開啟各資料夾人臉圖片1~10
        img_face = cv2.imread("att_faces/s" + str(folder_face_idx) + "/" + str(img_face_idx) + '.pgm')   #讀取圖片
        # print(img_face.shape)

        img_face = cv2.cvtColor(img_face, cv2.COLOR_BGR2GRAY)      #將圖片轉為灰階
        img_face_1D = img_face.flatten()                           #將二維轉成一維
        img_face_1D = np.array([img_face_1D])                      #轉成二維，才能用np.append放到二維array裡面
        img_face_10304x1 = img_face_1D
        #用np.append把一維array逐一放到二維array裡面
        img_face_10304x200 =  np.append(img_face_10304x200, img_face_10304x1,axis= 0)
        

img_face_10304x200 = img_face_10304x200[1:]

target_face = np.ones(shape = (img_face_10304x200.shape[0]),dtype=int)   #把"是人臉"的標籤存放到標籤的array

print("Image with Face Flatten DATA SIZE：" , img_face_10304x200.shape)
print("Image with Face Lable SIZA：", target_face.shape)




Image with Face Flatten DATA SIZE： (400, 10304)
Image with Face Lable SIZA： (400,)


In [18]:
'''讀取"不是人臉"pgm影像，並轉換成一維樣本集'''

#導入函式庫
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


img_unface_10304x200 = np.zeros(shape = (1,10304),dtype=int)   #建立要放入PCA的array

for img_unface_idx in range(200):                                            #選取開啟各資料夾人臉圖片1~10
    img_unface = cv2.imread("data_not_face/" + str(img_unface_idx) + '.jpg') #讀取圖片
    img_unface = cv2.cvtColor(img_unface, cv2.COLOR_BGR2GRAY)                #將圖片轉為灰階

    #可裁切的大小要大於要裁切的大小
    if (img_unface.shape[0] >= img_face.shape[0]) and (img_unface.shape[1] >= img_face.shape[1]):
        
        #設定裁切位置對齊圖片中央
        ori_pit_Y_of_cutted_unface = ((img_unface.shape[0]//2) - (img_face.shape[0]//2)) 
        ori_pit_X_of_cutted_unface = ((img_unface.shape[1]//2) - (img_face.shape[1]//2))

    #裁切圖片中央的 92*112 大小
    img_unface = img_unface[ori_pit_Y_of_cutted_unface:ori_pit_Y_of_cutted_unface+img_face.shape[1] , ori_pit_X_of_cutted_unface : ori_pit_X_of_cutted_unface+img_face.shape[0]]
    
    img_unface_1D = img_unface.flatten()      #將二維圖片轉換成一維
    img_unface_1D = np.array([img_unface_1D]) #轉成二維，才能用np.append放到二維array裡面
    img_unface_10304x1 = img_unface_1D
    
    #用np.append把一維array逐一放到二維array裡面
    img_unface_10304x200 =  np.append( img_unface_10304x200, img_unface_10304x1,axis= 0)

    
img_unface_10304x200 = img_unface_10304x200[1:]

target_unface = np.zeros(shape = (img_unface_10304x200.shape[0]),dtype=int)#把"不是人臉"的標籤存放到標籤的array


print("Image without Face Flatten DATA SIZE：" , img_unface_10304x200.shape)
print("Image without Face Lable Size：" , target_unface.shape)


Image without Face Flatten DATA SIZE： (200, 10304)
Image without Face Lable Size： (200,)


In [3]:
'''設定貝氏分類器'''
from sklearn.naive_bayes import GaussianNB

def DO_RECOG(Data_train , Data_test , Target_train , Target_test):

    model = GaussianNB()                     #建立Gaussian貝氏分類器模型
    model.fit(Data_train , Target_train)     #Training
    recog_anwser = model.predict(Data_test)  #Testing

    train_score = model.score(Data_train , Target_train)   #Training正確率
    test_score = model.score(Data_test , Target_test)      #Testing正確率

    print("訓練集正確率 = ",  train_score)
    print("測試集正確率 = ",  test_score)

    return recog_anwser , Target_test #回傳"判定結果"和"正確答案集"






In [4]:
'''設定混淆矩陣'''

from sklearn.metrics import confusion_matrix

def DO_Confusi_Matrix(correct_answer , recog_anwser , count_dimeniton):
    
    if count_dimeniton == 0:  #處裡單個辨識結果
        cur_dim_confus_mix = confusion_matrix(correct_answer, recog_anwser)
        print("----- Dimeniton : X   -----\n",cur_dim_confus_mix)
    
    else:
        for cur_dimeniton in range(5):    #處裡多個不同維度辨識結果
            cur_dim_confus_mix = confusion_matrix(correct_answer[cur_dimeniton], recog_anwser[cur_dimeniton])
            print("----- Dimeniton : "+str(cur_dimeniton+1) +"   -----\n", cur_dim_confus_mix)


In [5]:
'''設定PCA'''

import numpy as np
from sklearn.decomposition import PCA

def DO_PCA(Data_train , Data_test , set_dim):
    pca = PCA(n_components = set_dim)    #PCA降維維度
    pca.fit(Data_train)                  #降維train dataset
    pca.fit(Data_test)                   #降維test dataset
    transf_Data_train = pca.transform(Data_train)   #train dataset降維投影轉換
    transf_Data_test = pca.transform(Data_test)     #test dataset降維投影轉換

    return transf_Data_train , transf_Data_test  #回傳"轉換後train dataset"和"轉換後test dataset"

In [6]:
'''設定LDA'''

import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA


def DO_LDA(Data_train , Data_test , Label_train , set_dim):
    
    if set_dim == 0:     #不降維，只轉換
        lda = LDA()
        lda.fit(Data_train , Label_train)                   #訓練LDA train dataset
        transf_Data_train = lda.transform(Data_train)       #train dataset轉換投影
        transf_Data_test = lda.transform(Data_test)         #test dataset轉換投影
    
    else:                #降維，且轉換
        lda = LDA(n_components = set_dim)    #設定LDA降維維度
        lda.fit(Data_train , Label_train)    #訓練LDA train資料集
        transf_Data_train = lda.transform(Data_train)    #train dataset轉換投影
        transf_Data_test = lda.transform(Data_test)      #test dataset轉換投影
    
    return transf_Data_train,transf_Data_test #回傳"轉換後train dataset"和"轉換後test dataset"


In [20]:
'''圖片資料與標籤集分組'''

from sklearn.model_selection import train_test_split

Data =  np.append(img_face_10304x200 , img_unface_10304x200,axis= 0) #合併資料
Label = np.append(target_face , target_unface) #合併標籤

#分割訓練用的資料集
Data_train , Data_test , Label_train , Label_test = train_test_split(Data,Label,test_size = 0.3,stratify = Label) 

print("數據Data_train= " , Data_train.shape)
print("數據Data_test= " , Data_test.shape)
print("標籤Label_train= " , Label_train.shape)
print("標籤Label_test= " , Label_test.shape)



數據Data_train=  (420, 10304)
數據Data_test=  (180, 10304)
標籤Label_train=  (420,)
標籤Label_test=  (180,)


In [21]:
'''用PCA把2個數據集降成5種維度'''

Data_train_PCA_all_dim = [] #存放降維後train dataset PCA降維結果
Data_test_PCA_all_dim = []  #存放降維後test dataset PCA降維結果

for set_PCA_dimeniton in range(1,6):  #做10~50維度的降維

    Data_train_PCA , Data_test_PCA = DO_PCA(Data_train , Data_test , set_PCA_dimeniton*10) 


    print("降維後 Data_train =", Data_train_PCA.shape)
    print("降維後 Data_test =", Data_test_PCA.shape)

    Data_train_PCA_all_dim.append(Data_train_PCA) #收集降維後train dataset PCA降維結果
    Data_test_PCA_all_dim.append(Data_test_PCA)   #收集降維後train dataset PCA降維結果

降維後 Data_train = (420, 10)
降維後 Data_test = (180, 10)
降維後 Data_train = (420, 20)
降維後 Data_test = (180, 20)
降維後 Data_train = (420, 30)
降維後 Data_test = (180, 30)
降維後 Data_train = (420, 40)
降維後 Data_test = (180, 40)
降維後 Data_train = (420, 50)
降維後 Data_test = (180, 50)


In [22]:
'''使用PCA做5個降維資料的貝氏分類器'''

PCA_recog_ans = []    #存放經PCA降維辨識結果
PCA_correct_ans = []  #存放經PCA降維正確答案
for cur_dimeniton in range(5):
    
    #丟入貝氏分類器做分類判別
    cur_dim_recog_anwser , cur_dim_correct_answer = DO_RECOG(Data_train_PCA_all_dim[cur_dimeniton], Data_test_PCA_all_dim[cur_dimeniton], Label_train , Label_test)
    
    PCA_recog_ans.append(cur_dim_recog_anwser)     #收集經PCA降維辨識結果
    PCA_correct_ans.append(cur_dim_correct_answer) #收集經PCA降維正確答案


訓練集正確率 =  0.9809523809523809
測試集正確率 =  0.9666666666666667
訓練集正確率 =  0.9761904761904762
測試集正確率 =  0.9388888888888889
訓練集正確率 =  0.9452380952380952
測試集正確率 =  0.9277777777777778
訓練集正確率 =  0.9428571428571428
測試集正確率 =  0.9333333333333333
訓練集正確率 =  0.9452380952380952
測試集正確率 =  0.9277777777777778


In [23]:
'''做LDA轉換'''

Data_train_LDA_all_dim = []  #存放LDA降維後train dataset
Data_test_LDA_all_dim = []   #存放LDA降維後test dataset

set_LDA_dimeniton = 0

#丟進LDA做轉換，得到train dataset LDA轉換和test dataset LDA轉換的結果
Data_train_LDA , Data_test_LDA = DO_LDA(Data_train , Data_test , Label_train , set_LDA_dimeniton*10)
print("轉換後Data_train =", Data_train_LDA.shape)
print("轉換後Data_test =", Data_test_LDA.shape)



轉換後Data_train = (420, 1)
轉換後Data_test = (180, 1)


In [24]:
'''做 LDA 轉換資料的貝斯分類器'''

#丟入貝氏分類器做分類判別
LDA_recog_ans , LDA_correct_ans = DO_RECOG(Data_train_LDA , Data_test_LDA, Label_train , Label_test)

訓練集正確率 =  0.9976190476190476
測試集正確率 =  0.9


In [25]:
'''做混淆矩陣'''

print("PCA 混淆矩陣")
DO_Confusi_Matrix(PCA_correct_ans , PCA_recog_ans , set_PCA_dimeniton)  
print("LDA 混淆矩陣")
DO_Confusi_Matrix(LDA_correct_ans , LDA_recog_ans , set_LDA_dimeniton)

PCA 混淆矩陣
----- Dimeniton : 1   -----
 [[ 54   6]
 [  0 120]]
----- Dimeniton : 2   -----
 [[ 49  11]
 [  0 120]]
----- Dimeniton : 3   -----
 [[ 47  13]
 [  0 120]]
----- Dimeniton : 4   -----
 [[ 48  12]
 [  0 120]]
----- Dimeniton : 5   -----
 [[ 47  13]
 [  0 120]]
LDA 混淆矩陣
----- Dimeniton : X   -----
 [[ 42  18]
 [  0 120]]
